In [ ]:
import os
import urllib.request
import cv2
import numpy as np
from ultralytics import YOLO

# -------------------- MODEL AUTO DOWNLOAD --------------------
model_path = "yolo11s.pt"
if not os.path.exists(model_path):
    print("Downloading YOLO11 model...")
    url = "https://github.com/ultralytics/assets/releases/download/v8.1.0/yolov8s.pt"
    urllib.request.urlretrieve(url, model_path)
    print("Model Downloaded!")

model = YOLO(model_path)
print("✔ YOLO Model Loaded")

# -------------------- YOUR VIDEO (same location) --------------------
video_path = "road_trafifc.mp4"

if not os.path.exists(video_path):
    raise FileNotFoundError("❌ road_trafifc.mp4 not found! Put it in the SAME location as main.py")

# -------------------- LANE DETECTION --------------------
def detect_lanes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 100, 200)

    h, w = edges.shape
    mask = np.zeros_like(edges)

    roi = np.array([[(0,h), (w,h), (w,int(h*0.55)), (0,int(h*0.55))]])
    cv2.fillPoly(mask, roi, 255)

    masked = cv2.bitwise_and(edges, mask)

    lines = cv2.HoughLinesP(masked, 1, np.pi/180, 70, minLineLength=80, maxLineGap=60)
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line[0]
            cv2.line(frame, (x1,y1), (x2,y2), (0,255,0), 3)

    return frame

# -------------------- SPEED DETECTION --------------------
previous_positions = {}
pixel_dist = 80
meters_dist = 8
fps = 24
speed_factor = (meters_dist / pixel_dist) * fps * 3.6   # km/h

# -------------------- VIDEO PROCESSING --------------------
cap = cv2.VideoCapture(video_path)

# Output video (save in SAME location, no folder)
output_name = "output_result.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_name, fourcc, 24,
                      (int(cap.get(3)), int(cap.get(4))))

print("✔ Processing Started... Press Q to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("✔ Video Finished!")
        break

    results = model(frame, stream=True)

    for r in results:
        boxes = r.boxes.xyxy
        ids = r.boxes.id
        cls = r.boxes.cls

        if ids is None:
            continue

        for i, box in enumerate(boxes):
            x1,y1,x2,y2 = map(int, box)
            obj_id = ids[i].item()
            label = int(cls[i])

            if label not in [2,3,5,7]:  # car, bike, bus, truck
                continue

            cx = int((x1 + x2)//2)
            cy = int((y1 + y2)//2)

            # SPEED CALCULATION
            if obj_id in previous_positions:
                old_x, old_y = previous_positions[obj_id]
                movement = abs(cx - old_x)
                speed = movement * speed_factor
                cv2.putText(frame, f"{speed:.1f} km/h",
                            (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                            0.7, (255,50,0), 2)

            previous_positions[obj_id] = (cx, cy)

            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,120,255), 2)
            cv2.circle(frame, (cx,cy), 5, (0,255,255), -1)

    # LANE DETECTION
    frame = detect_lanes(frame)

    out.write(frame)

    cv2.imshow("YOLO11 Traffic Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✔ Output saved: {output_name}")
print("✔ Traffic Analysis Done!")


✔ YOLO11 Model Loaded


FileNotFoundError: Your video road_trafifc.mp4 not found!

In [ ]:
!pip install ultralytics 


     ---------------------------------------- 1.1/1.1 MB 3.6 MB/s eta 0:00:00
     -------------------------------------- 783.6/783.6 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 12.9/12.9 MB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 14.4 MB/s eta 0:00:00
     ---------------------------------------- 8.1/8.1 MB 7.9 MB/s eta 0:00:00
     -------------------------------------- 158.6/158.6 kB 4.8 MB/s eta 0:00:00
     ------------------------------------- 111.0/111.0 MB 11.5 MB/s eta 0:00:00
     --------------------------------------- 39.0/39.0 MB 15.6 MB/s eta 0:00:00
     --------------------------------------- 41.3/41.3 MB 13.6 MB/s eta 0:00:00
     ---------------------------------------- 64.7/64.7 kB ? eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 18.2 MB/s eta 0:00:00
     -------------------------------------- 113.9/113.9 kB 6.9 MB/s eta 0:00:00
     -------------------------------------- 221.2/221


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
